In [ ]:
!pip install folium

In [ ]:
%matplotlib inline

In [ ]:
%%html
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-button/paper-button.html" is='urth-core-import' package='PolymerElements/paper-button'>
<link rel="import" href="urth_components/paper-card/paper-card.html" is='urth-core-import' package='PolymerElements/paper-card'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel="import" href="urth_components/google-map/google-map.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/google-map/google-map-marker.html" is='urth-core-import' package='GoogleWebComponents/google-map'>
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is='urth-core-import'>
<link rel="import" href="urth_components/urth-viz-chart/urth-viz-chart.html" is='urth-core-import'>
<link rel="stylesheet" href="http://cdn.leafletjs.com/leaflet-0.5/leaflet.css" />
<link rel="import" href="./urth-raw-html.html">

In [ ]:
import os
import struct
import glob
import pandas as pd
import datetime as dt

# Use this global variable to specify the path for station summary files.
NOAA_STATION_SUMMARY_PATH = "/home/jovyan/work/noaa/data/"

# Use this global variable to specify the path for the GHCND Station Directory
STATION_DETAIL_FILE = '/home/jovyan/work/noaa/data/ghcnd-stations.txt'

# Station detail structures for building station lists
station_detail_colnames = ['StationID','State','Name',
                            'Latitude','Longitude','QueryTag']

station_detail_rec_template = {'StationID': "",
                                'State': "",
                                'Name': "",
                                'Latitude': "",
                                'Longitude': "",
                                'QueryTag': ""
                                }

# -----------------------------------
# Station Detail Processing
# -----------------------------------
def get_filename(pathname):
    '''Fetch filename portion of pathname.'''
    plist = pathname.split('/')
    fname, fext = os.path.splitext(plist[len(plist)-1])
    return fname

def fetch_station_list():
    '''Return list of available stations given collection of summary files on disk.'''
    station_list = []
    raw_files = os.path.join(NOAA_STATION_SUMMARY_PATH,'','*_sum.csv')
    for index, fname in enumerate(glob.glob(raw_files)):
        f = get_filename(fname).split('_')[0]
        station_list.append(str(f))
    return station_list

USA_STATION_LIST = fetch_station_list()

def gather_states(fname,stations): 
    '''Return a list of unique State abbreviations. Weather station data exists for these states.'''
    state_list = []
    with open(fname, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        f.close()
        for line in lines:
            r = noaa_gather_station_detail(line,stations)
            state_list += r
    df_unique_states = pd.DataFrame(state_list,columns=station_detail_colnames).sort('State').State.unique()
    return df_unique_states.tolist()

def noaa_gather_station_detail(line,slist):
    '''Build a list of station tuples for stations in the USA.'''
    station_tuple_list = []
    station_id_key = line[0:3]
    if station_id_key == 'USC' or station_id_key == 'USW': 
        fields = struct.unpack('12s9s10s7s2s30s', line[0:70].encode())
        if fields[0].decode().strip() in slist:
            station_tuple = dict(station_detail_rec_template)
            station_tuple['StationID'] = fields[0].decode().strip()
            station_tuple['State'] = fields[4].decode().strip()
            station_tuple['Name'] = fields[5].decode().strip()
            station_tuple['Latitude'] = fields[1].decode().strip()
            station_tuple['Longitude'] = fields[2].decode().strip()
            qt = "{0} at {1} in {2}".format(fields[0].decode().strip(),fields[5].decode().strip(),fields[4].decode().strip())
            station_tuple['QueryTag'] = qt
            station_tuple_list.append(station_tuple)
    return station_tuple_list

USA_STATES_WITH_STATIONS = gather_states(STATION_DETAIL_FILE,USA_STATION_LIST)

def process_station_detail_for_state(fname,stations,statecode): 
    '''Return dataframe of station detail for specified state.'''
    station_list = []
    with open(fname, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        f.close()
        for line in lines:
            r = noaa_build_station_detail_for_state(line,stations,statecode)
            station_list += r
    return pd.DataFrame(station_list,columns=station_detail_colnames)

def noaa_build_station_detail_for_state(line,slist,statecode):
    '''Build a list of station tuples for the specified state in the USA.'''
    station_tuple_list = []
    station_id_key = line[0:3]
    if station_id_key == 'USC' or station_id_key == 'USW':
        fields = struct.unpack('12s9s10s7s2s30s', line[0:70].encode())
        if ((fields[0].decode().strip() in slist) and (fields[4].decode().strip() == statecode)): 
            station_tuple = dict(station_detail_rec_template)
            station_tuple['StationID'] = fields[0].decode().strip()
            station_tuple['State'] = fields[4].decode().strip()
            station_tuple['Name'] = fields[5].decode().strip()
            station_tuple['Latitude'] = fields[1].decode().strip()
            station_tuple['Longitude'] = fields[2].decode().strip()
            qt = "Station {0} in {1} at {2}".format(fields[0].decode().strip(),fields[4].decode().strip(),fields[5].decode().strip())
            station_tuple['QueryTag'] = qt
            station_tuple_list.append(station_tuple)
    return station_tuple_list

df = process_station_detail_for_state(STATION_DETAIL_FILE,USA_STATION_LIST,"NY")
df.tail()

In [ ]:
import numpy as np
import folium
from IPython.display import HTML

def display_map(m, height=500):
    """Takes a folium instance and embed HTML."""
    m._build_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = '<iframe srcdoc="{0}" style="width: 100%; height: {1}px; border: none"></iframe>'.format(srcdoc, height)
    return embed

def render_map(df,height=500):
    centerpoint_latitude = np.mean(df.Latitude.astype(float))
    centerpoint_longitude = np.mean(df.Longitude.astype(float))
    map_obj = folium.Map(location=[centerpoint_latitude, centerpoint_longitude],zoom_start=6)
    for index, row in df.iterrows():
        map_obj.simple_marker([row.Latitude, row.Longitude], popup=row.QueryTag)
    return display_map(map_obj)

In [ ]:
map_doc = render_map(df)

In [ ]:
from urth.widgets.widget_channels import channel
channel("noaaquery").set("theMap", map_doc)

In [ ]:
%%html
<template id="narrationContent" is="urth-core-bind" channel="noaaquery">
    <div id="map">
        <urth-raw-html html="{{theMap}}"/>
    </div>
</template>

In [ ]:
map_doc